<a href="https://colab.research.google.com/github/ferrry-ghsb/bigdata_6team/blob/main/RNN_%EB%9E%A8_%EB%B6%80%EC%A1%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import io
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_rows = 50
pd.options.display.max_columns = 40
import numpy as np
from sklearn import preprocessing

In [2]:
from keras.models import Sequential, Model, load_model
from keras.layers import Input, Dense, Activation, Flatten, Dropout
from keras.layers import SimpleRNN

In [3]:
x_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/x_train.csv')
x_valid = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/x_valid.csv')
y_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/y_train.csv')
y_valid = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/y_valid.csv')

In [4]:
x_train = x_train[x_train.date >= '2016-01-01']
x_valid = x_valid[x_valid.date <= '2016-09-12']
y_train = x_train[x_train.date >= '2016-01-01']
y_valid = x_valid[x_valid.date <= '2016-09-12']

In [5]:
x_train = x_train.replace('-', '', regex=True).astype(int)
x_valid = x_valid.replace('-', '', regex=True).astype(int)
y_train = y_train.replace('-', '', regex=True).astype(int)
y_valid = y_valid.replace('-', '', regex=True).astype(int)

In [6]:
scaler_X_tr = preprocessing.MinMaxScaler()
scaler_Y_tr = preprocessing.MinMaxScaler()
sequence = 34
batch_size = 32
epoch = 20
verbose = 1
dropout_ratio = 0

In [7]:
x_train['date']

1972674    20160101
1972675    20160101
1972676    20160101
1972677    20160101
1972678    20160101
             ...   
2443473    20160912
2443474    20160912
2443475    20160912
2443476    20160912
2443477    20160912
Name: date, Length: 470804, dtype: int64

In [8]:
X_train_scaled = scaler_X_tr.fit_transform(x_train)
Y_train_scaled = scaler_Y_tr.fit_transform(y_train)
X_valid_scaled = scaler_X_tr.transform(x_valid)
Y_valid_scaled = scaler_Y_tr.transform(y_valid)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:488: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:488: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


In [ ]:
# 여기서부터 램 용량 부족..
# X / Y split

x_train, y_train = [], []
for index in range(len(X_train_scaled) - sequence):
    x_train.append(np.array(X_train_scaled[index: index + sequence]))
    y_train.append(np.ravel(Y_train_scaled[index + sequence:index + sequence + 1]))
x_train, y_train = np.array(x_train), np.array(y_train)
x_valid, y_valid = [], []
for index in range(len(X_valid_scaled) - sequence):
    x_valid.append(np.array(X_valid_scaled[index: index + sequence]))
    y_valid.append(np.ravel(Y_valid_scaled[index + sequence:index + sequence + 1]))  
x_valid, y_valid = np.array(x_valid), np.array(y_valid)

In [ ]:
# RNN
model = Sequential()
model.add(SimpleRNN(128, input_shape=(x_train.shape[1], x_train.shape[2]), return_sequences=True, activation='relu'))
model.add(Dropout(dropout_ratio))
# 램 부족으로 일단은 64의 경우만 실행
# model.add(SimpleRNN(256, return_sequences=True, activation="relu"))
# model.add(Dropout(dropout_ratio))
# model.add(SimpleRNN(128, return_sequences=True, activation="relu"))
# model.add(Dropout(dropout_ratio)) 
model.add(SimpleRNN(64, return_sequences=True, activation="relu"))
model.add(Dropout(dropout_ratio)) 
model.add(Flatten())
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()
model_fit = model.fit(X_train, Y_train, 
                      batch_size=batch_size, epochs=epoch,
                      verbose=verbose)

plt.plot(pd.DataFrame(model_fit.history))
plt.grid(True)
plt.show()